<a href="https://colab.research.google.com/github/adampotton/MDM3-Rep-3/blob/main/S2_CNN_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import files, drive
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


#### Load csv of the amount of tree in each sample and choose classifciation mode

In [ ]:

table = pd.read_csv('/content/drive/My Drive/S2 Data/labels.csv')
mapping = pd.read_csv('/content/drive/My Drive/S2 Data/tree_mapping.csv')
df = pd.DataFrame(mapping)

# 0 = species level (19 classes)
# 1 = genus level (14 classes)
# 2 = stand type (9 classes)
# 3 = foliage type (2 classes)
classification = 1



#### Load NumPy arrays and convert labels array to a numbered labels array


In [ ]:
train_images = np.load('/content/drive/My Drive/S2 Data/train_images.npy')
train_names = np.load('/content/drive/My Drive/S2 Data/train_labels.npy')
test_images = np.load('/content/drive/My Drive/S2 Data/test_images.npy')
test_names = np.load('/content/drive/My Drive/S2 Data/test_labels.npy')
val_images = np.load('/content/drive/My Drive/S2 Data/val_images.npy')
val_names = np.load('/content/drive/My Drive/S2 Data/val_labels.npy')

def map_tree_name(name, classification):
    name = name.replace('_', ' ')
    row_number = df.index[df['Species'] == name][0]
    if classification == 0:
        return name
    elif classification == 1:
        return df.iloc[row_number, 1]
    elif classification == 2:
        return df.iloc[row_number, 2]
    elif classification == 3:
        return df.iloc[row_number, 3]

def apply_mapping(names, classification):
    return np.array([map_tree_name(name, classification) for name in names])

train_names2 = apply_mapping(train_names, classification)
test_names2 = apply_mapping(test_names, classification)
val_names2 = apply_mapping(val_names, classification)


tree_names, train_labels = np.unique(train_names2, return_inverse=True)
tree_names, test_labels = np.unique(test_names2, return_inverse=True)
tree_names, val_labels = np.unique(val_names2, return_inverse=True)

train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)
val_images = val_images.astype(np.float32)


mean = np.mean(np.concatenate([train_images, test_images, val_images], axis=0), axis=(0, 1, 2))
std = np.std(np.concatenate([train_images, test_images, val_images], axis=0), axis=(0, 1, 2))

train_images = (train_images - mean) / std
test_images = (test_images - mean) / std
val_images = (val_images - mean) / std


#### Create Data Loaders


In [ ]:
# Convert NumPy arrays to PyTorch tensors
train_images_tensor = torch.tensor(train_images, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)
val_images_tensor = torch.tensor(val_images, dtype=torch.float32)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)

batch_size = 64

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

#### Define the Model, and other paramaeters

In [ ]:

num_channels = 12
num_classes = len(tree_names)
learning_rate = 0.001

model = nn.Sequential(
    nn.Conv2d(num_channels, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),

    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64 * 1 * 1, 256),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.00005)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=2)

#### Train, test, and validate, plotting validation loss

In [1]:
def train():
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images.permute(0, 3, 1, 2))

        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def validation(epoch):
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images.permute(0, 3, 1, 2))

            loss = criterion(logits, labels)
            val_loss += loss.item()

            predicted = torch.argmax(logits, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            accuracy = 100 * correct / total

        val_loss /= len(test_loader)
        scheduler.step(val_loss)

        if (epoch + 1) % 5 == 0:
            print(f'Validation accuracy after {epoch+1} epochs: {accuracy:.2f}%')
    return val_loss

def test():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images.permute(0, 3, 1, 2))

            predicted = torch.argmax(logits, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            accuracy = 100 * correct / total
        print(f'Test accuracy {100 * correct / total:.2f}%')

num_epochs = 100
val_losses = []
for epoch in range(num_epochs):
    train()
    val_loss = validation(epoch)
    val_losses.append(val_loss)
test()

plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss over Epochs')
plt.legend()
plt.show()

NameError: name 'model' is not defined